In [2]:
from api_call import *
api_options = ApiCall(data_type='Options')
api_spot = ApiCall(data_type='Equity')
api_expiry = ApiCall(data_type='ExpiryDates')

ModuleNotFoundError: No module named 'api_call'

In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime, timedelta
import pandas as pd
from tkcalendar import DateEntry
import plotly.graph_objects as go
import plotly.io as pio
import tempfile
import webbrowser
import os

class OptionsChartGUI:
    def __init__(self, root):
        self.root = root
        self.root.title('Options Data Fetcher with Chart')

        # Load expiry dates
        try:
            df = pd.read_excel('NIFTY_Expiry_Dates.xlsx')
            expiry_series = pd.to_datetime(
                df.iloc[:, 0].dropna().astype(str),
                dayfirst=True,
                errors='coerce'
            ).dropna()
            self.expiry_dates_list = expiry_series.tolist()
            self.expiry_dates_list.sort()
        except Exception as e:
            messagebox.showerror('Error', f'Failed to load expiry dates: {str(e)}')
            self.expiry_dates_list = []

        # Main frame (UI elements)
        main_frame = ttk.Frame(root, padding='10')
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E))

        # Start Date
        ttk.Label(main_frame, text='Start Date:').grid(row=0, column=0, padx=5, pady=5)
        self.start_date = DateEntry(main_frame, width=15, background='darkblue',
                                    foreground='white', date_pattern='dd-mm-yyyy')
        self.start_date.grid(row=0, column=1, padx=5, pady=5)
        self.start_date.bind('<<DateEntrySelected>>', self.auto_set_expiry)

        # End Date
        ttk.Label(main_frame, text='End Date:').grid(row=0, column=2, padx=5, pady=5)
        self.end_date = ttk.Entry(main_frame, width=15, state='readonly')
        self.end_date.grid(row=0, column=3, padx=5, pady=5)

        # Expiry Date
        ttk.Label(main_frame, text='Expiry Date:').grid(row=0, column=4, padx=5, pady=5)
        self.expiry_date = ttk.Entry(main_frame, width=15, state='readonly')
        self.expiry_date.grid(row=0, column=5, padx=5, pady=5)

        # Strike Price
        ttk.Label(main_frame, text='Strike Price:').grid(row=0, column=6, padx=5, pady=5)
        self.strike_price = ttk.Entry(main_frame, width=10)
        self.strike_price.grid(row=0, column=7, padx=5, pady=5)

        # Option Type
        ttk.Label(main_frame, text='Option Type:').grid(row=0, column=8, padx=5, pady=5)
        self.option_type = ttk.Combobox(main_frame, values=['CE', 'PE'], state='readonly', width=5)
        self.option_type.set('CE')
        self.option_type.grid(row=0, column=9, padx=5, pady=5)

        # Interval
        ttk.Label(main_frame, text='Interval:').grid(row=0, column=10, padx=5, pady=5)
        self.time_interval = ttk.Combobox(
            main_frame, values=['1min', '5min', '15min', '30min', '1h', '1D'], state='readonly', width=6
        )
        self.time_interval.set('15min')
        self.time_interval.grid(row=0, column=11, padx=5, pady=5)

        # Download Button
        ttk.Button(main_frame, text='Download Data', command=self.fetch_and_plot).grid(
            row=0, column=12, padx=10, pady=5
        )

        # Chart Frame placeholder (not used for browser plot but kept for future in-app embedding)
        self.chart_frame = ttk.Frame(root, padding='10')
        self.chart_frame.grid(row=1, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

    def auto_set_expiry(self, event=None):
        try:
            selected_date = self.start_date.get_date()
            expiry_dates = [pd.to_datetime(exp).date() for exp in self.expiry_dates_list]
            nearest_expiry = next((exp for exp in expiry_dates if exp >= selected_date), None)

            if nearest_expiry:
                expiry_str = nearest_expiry.strftime('%d-%m-%Y')
                self.expiry_date.config(state='normal')
                self.expiry_date.delete(0, tk.END)
                self.expiry_date.insert(0, expiry_str)
                self.expiry_date.config(state='readonly')

                self.end_date.config(state='normal')
                self.end_date.delete(0, tk.END)
                self.end_date.insert(0, expiry_str)
                self.end_date.config(state='readonly')
            else:
                messagebox.showwarning('Warning', 'No expiry date found')
        except Exception as e:
            messagebox.showerror('Expiry Error', str(e))

    def fetch_and_plot(self):
        try:
            start_dt = self.start_date.get_date().strftime('%d-%m-%Y')
            end_dt = self.end_date.get()
            expiry = self.expiry_date.get()
            strike_text = self.strike_price.get().strip()
            if strike_text == '':
                messagebox.showwarning('Input', 'Enter Strike Price')
                return
            strike_price = int(float(strike_text))
            option_type = self.option_type.get()
            interval = self.time_interval.get()

            # API call
            api_options = ApiCall(data_type='Options')
            data = api_options.get_data(
                symbol=['NIFTY'],
                start_date=start_dt,
                end_date=end_dt,
                start_expiry=expiry,
                strike_price=[strike_price],
                option_type=option_type,
                resample_bool=True,
                resample_freq=interval,
                label='left',
            )

            if data is None or getattr(data, 'empty', True):
                messagebox.showwarning('Warning', 'No data found')
                return

            # Render interactive Plotly chart in browser — pass strike and option type
            self.show_plotly_chart(data, strike_price=strike_price, option_type=option_type)

        except Exception as e:
            messagebox.showerror('Error', str(e))

    def show_plotly_chart(self, data: pd.DataFrame, strike_price=None, option_type=None):
        try:
            df = data.copy()
            # ensure datetime column
            if 'Datetime' not in df.columns:
                df['Datetime'] = pd.to_datetime(df.index)
            else:
                df['Datetime'] = pd.to_datetime(df['Datetime'])
            df = df.sort_values('Datetime').reset_index(drop=True)

            # ensure numeric OHLCV
            for col in ['Open', 'High', 'Low', 'Close']:
                if col not in df.columns:
                    raise ValueError(f"Missing required column: {col}")
                df[col] = pd.to_numeric(df[col], errors='coerce')
            df['Volume'] = pd.to_numeric(df.get('Volume', 0), errors='coerce').fillna(0)

            if len(df) == 0:
                messagebox.showwarning('Warning', 'No data to plot')
                return

            # Identify full calendar days that have no data between min and max date
            start_date = df['Datetime'].dt.date.min()
            end_date = df['Datetime'].dt.date.max()
            all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
            present_dates = set(df['Datetime'].dt.date.unique())
            missing_full_days = [d.strftime('%Y-%m-%d') for d in all_dates.date if d not in present_dates]

            # Rangebreaks: always remove weekends and non-trading hours (15:15 -> 09:15) to remove intraday gap
            rangebreaks = [
                dict(bounds=['sat', 'mon']),
                dict(pattern='hour', bounds=['15:15', '09:15']),
            ]
            if missing_full_days:
                MAX_DAY_EXCLUDE = 1000
                if len(missing_full_days) <= MAX_DAY_EXCLUDE:
                    rangebreaks.append(dict(values=missing_full_days))

            # Prepare day separators for visual clarity (start of each present trading day)
            day_first = df.groupby(df['Datetime'].dt.date)['Datetime'].first().tolist()
            shapes = []
            for d in day_first[1:]:
                shapes.append({
                    'type': 'line',
                    'x0': d.isoformat(),
                    'x1': d.isoformat(),
                    'xref': 'x',
                    'yref': 'paper',
                    'y0': 0,
                    'y1': 1,
                    'line': {'color': '#444', 'width': 1, 'dash': 'dash'}
                })

            # Build plotly traces
            candlestick = go.Candlestick(
                x=df['Datetime'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name='Price',
            )
            volume = go.Bar(
                x=df['Datetime'],
                y=df['Volume'],
                marker_color='rgba(100,100,100,0.5)',
                name='Volume',
                yaxis='y2',
            )

            # Build title from passed parameters (fallback to generic)
            title_text = f"{strike_price} {option_type}" if (strike_price is not None and option_type) else "Options Chart"

            # Use the exact xaxis logic requested by user
            layout = go.Layout(
                title=title_text,
                template='plotly_dark',
                hovermode='x unified',
                dragmode='pan',
                xaxis=dict(
                    rangeslider=dict(visible=False),
                    rangebreaks=[
                        dict(bounds=["sat", "mon"]),       # weekends
                        dict(pattern="hour", bounds=[15.5, 9.25])  # trading hours only
                    ],
                    type='date',
                    showgrid=False,
                    tickformat='%H:%M\n%b %d',
                    range=[df['Datetime'].min(), df['Datetime'].max()]
                ),
                yaxis=dict(domain=[0.3, 1.0], title='Price', autorange=True, fixedrange=False),
                yaxis2=dict(domain=[0.0, 0.2], title='Volume', anchor='x', showgrid=False, fixedrange=False),
                shapes=shapes,
                legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
            )

            fig = go.Figure(data=[candlestick, volume], layout=layout)

            # write to temp HTML and open in default browser with scrollZoom enabled
            tmp = tempfile.NamedTemporaryFile(delete=False, suffix='.html')
            pio.write_html(fig, file=tmp.name, auto_open=False, full_html=True, config={'scrollZoom': True})
            webbrowser.open('file://' + os.path.realpath(tmp.name))

        except Exception as e:
            messagebox.showerror('Chart Error', str(e))

    def plot_in_tk(self, data: pd.DataFrame, strike_price, option_type):
        try:
            df = data.copy()
            # ensure Datetime and proper types
            if 'Datetime' not in df.columns:
                df['Datetime'] = pd.to_datetime(df.index)
            else:
                df['Datetime'] = pd.to_datetime(df['Datetime'])
            df = df.sort_values('Datetime').reset_index(drop=True)

            # required columns
            for col in ['Open', 'High', 'Low', 'Close']:
                if col not in df.columns:
                    raise ValueError(f'Missing required column: {col}')
                df[col] = pd.to_numeric(df[col], errors='coerce')
            df['Volume'] = pd.to_numeric(df.get('Volume', 0), errors='coerce').fillna(0)

            # prepare DataFrame for mplfinance (Datetime index)
            mpf_df = df.set_index('Datetime')[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

            # destroy previous canvas / toolbar (if any)
            if self.canvas:
                try:
                    self.canvas.get_tk_widget().destroy()
                except Exception:
                    pass
            if self.toolbar:
                try:
                    self.toolbar.destroy()
                except Exception:
                    pass

            # create mplfinance figure and axes
            fig, axes = mpf.plot(mpf_df, type='candle', volume=True, style='yahoo', returnfig=True, figsize=(10,6))
            price_ax = axes[0]               # main price axis
            vol_ax = axes[-1]                # robust volume axis reference (last axis)

            # set title to "strike option_type"
            fig.suptitle(f'{strike_price} {option_type}', fontsize=12)

            # force figure to reasonable size and remove x-axis margins so plot fills area
            try:
                fig.set_size_inches(12, 6)
                fig.tight_layout(rect=[0, 0.03, 1, 0.95])
            except Exception:
                pass
            try:
                price_ax.set_xlim(mpf_df.index[0], mpf_df.index[-1])
                price_ax.margins(x=0)
                price_ax.xaxis_date()
                fig.autofmt_xdate()
            except Exception:
                pass

            # increase price-axis tick density (more points)
            ymin, ymax = price_ax.get_ylim()
            tick_count = 18  # increase for more ticks (adjust as needed)
            raw_step = (ymax - ymin) / tick_count if (ymax - ymin) > 0 else 1.0
            # use raw_step directly to ensure more granular ticks
            price_ax.yaxis.set_major_locator(MultipleLocator(raw_step))

            # embed figure into Tkinter frame; ensure it expands to fill space
            self.canvas = FigureCanvasTkAgg(fig, master=self.chart_frame)
            widget = self.canvas.get_tk_widget()
            widget.grid(row=0, column=0, sticky='nsew')
            self.canvas.draw()

            # add toolbar below the canvas
            self.toolbar = NavigationToolbar2Tk(self.canvas, self.chart_frame)
            self.toolbar.update()
            self.toolbar.grid(row=1, column=0, sticky='ew')

            # create crosshair lines and small circular marker
            mpd0 = mdates.date2num(mpf_df.index[0])
            self.vline_price = price_ax.axvline(x=mpd0, color='white', linestyle='--', linewidth=0.8, alpha=0.8)
            self.vline_vol = vol_ax.axvline(x=mpd0, color='white', linestyle='--', linewidth=0.8, alpha=0.8)
            self.hline = price_ax.axhline(y=(ymin + ymax) / 2, color='white', linestyle='--', linewidth=0.8, alpha=0.8)

            # marker: small filled circle at intersection, hidden initially
            (self.marker_line,) = price_ax.plot([], [], marker='o', markersize=6, markeredgecolor='white',
                                            markerfacecolor='deepskyblue', zorder=12, linestyle='None', visible=False)

            # annotation box on price axis showing time and price
            self.annotation = price_ax.annotate('', xy=(0,0), xytext=(10,10), textcoords='offset points',
                                            color='white', bbox=dict(facecolor='black', alpha=0.7), fontsize=8)
            self.annotation.set_visible(False)

            # event handler to move crosshair and marker
            def on_move(event):
                if event.inaxes in (price_ax, vol_ax):
                    xdata = event.xdata
                    ydata = event.ydata if event.inaxes is price_ax else None
                    if xdata is None:
                        return
                    try:
                        self.vline_price.set_xdata(xdata)
                        self.vline_vol.set_xdata(xdata)
                    except Exception:
                        pass
                    if ydata is not None:
                        try:
                            self.hline.set_ydata(ydata)
                        except Exception:
                            pass
                        # update marker position and make it visible
                        try:
                            self.marker_line.set_data([xdata], [ydata])
                            self.marker_line.set_visible(True)
                        except Exception:
                            pass
                    else:
                        # hide marker if over volume axis (no price y)
                        try:
                            self.marker_line.set_visible(False)
                        except Exception:
                            pass

                    # datetime string
                    try:
                        dt = mdates.num2date(xdata)
                        tstr = dt.strftime('%Y-%m-%d %H:%M:%S')
                    except Exception:
                        tstr = str(xdata)

                    price_text = f'{ydata:.2f}' if ydata is not None else ''
                    ann_y = ydata if ydata is not None else (ymin + ymax) / 2
                    self.annotation.xy = (xdata, ann_y)
                    self.annotation.set_text(f'{tstr}\\n{price_text}')
                    self.annotation.set_visible(True)

                    self.canvas.draw_idle()
                else:
                    if self.annotation and self.annotation.get_visible():
                        self.annotation.set_visible(False)
                    try:
                        self.marker_line.set_visible(False)
                    except Exception:
                        pass
                    self.canvas.draw_idle()

            fig.canvas.mpl_connect('motion_notify_event', on_move)

        except Exception as e:
            messagebox.showerror('Chart Error', str(e))

    # ...existing other methods ...


In [3]:
# Launcher cell: run this cell (after running the ApiCall/imports cell and the cell defining OptionsChartGUI)
try:
    root = tk.Tk()
    app = OptionsChartGUI(root)
    root.mainloop()
except Exception as e:
    import traceback
    traceback.print_exc()

C:\Users\mca\AppData\Local\Temp\ipykernel_18400\3045687271.py:20: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  expiry_series = pd.to_datetime(
